### 1.	Выбор начальных условий

**a. Выбор набора данных для задачи классификации**

Для задачи классификации выбран набор данных **Fashion-MNIST**. Этот выбор обоснован тем, что набор данных представляет собой реальную практическую задачу классификации изображений одежды, что может быть полезно для разработки приложений в области электронной коммерции или автоматизации процессов на складах.

**b. Выбор метрик качества**

Для оценки качества моделей классификации выбраны следующие метрики:

1. **Точность (Accuracy):** доля правильно классифицированных примеров среди всех рассмотренных. Эта метрика подходит для задач с примерно одинаковым количеством примеров в каждом классе.

2. **F1-мера:** гармоническое среднее между точностью (precision) и полнотой (recall). Эта метрика полезна, когда классы в данных несбалансированы.

### 2. Создание бейзлайна и оценка качества

**a/b. Обучение и оценка качество моделей по выбранным метрикам на выбранном наборе данных**

Для обучения моделей будем использовать библиотеку PyTorch и torchvision.

In [34]:
import torch
import numpy as np
import torch.nn as nn
import torchvision
from torch.utils.data import Subset
import torchvision.transforms as transforms
from torchmetrics.classification import MulticlassAccuracy, MulticlassF1Score

device = torch.device("mps")
BATCH_SIZE = 128
NUM_CLASSES = 10
EPOCHS = 3
LR = 1e-4

transform_cnn = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
transform_vit = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.repeat(3, 1, 1)),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset_cnn = torchvision.datasets.FashionMNIST('data', train=True, download=True, transform=transform_cnn)
testset_cnn  = torchvision.datasets.FashionMNIST('data', train=False, download=True, transform=transform_cnn)
trainset_vit = torchvision.datasets.FashionMNIST('data', train=True, download=True, transform=transform_vit)
testset_vit  = torchvision.datasets.FashionMNIST('data', train=False, download=True, transform=transform_vit)


trainloader_cnn = torch.utils.data.DataLoader(trainset_cnn, batch_size=BATCH_SIZE, shuffle=True)
testloader_cnn  = torch.utils.data.DataLoader(testset_cnn, batch_size=BATCH_SIZE, shuffle=False)

N_SUBSET = 5000
np.random.seed(42)
indices_train = np.random.choice(len(trainset_vit), N_SUBSET, replace=False)
trainset_vit_small = Subset(trainset_vit, indices_train)

trainloader_vit = torch.utils.data.DataLoader(trainset_vit_small, batch_size=BATCH_SIZE, shuffle=True)
testloader_vit  = torch.utils.data.DataLoader(testset_vit, batch_size=BATCH_SIZE, shuffle=False)

# Модель CNN
cnn = torchvision.models.resnet18(weights=None, num_classes=NUM_CLASSES)
cnn.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
cnn.to(device)

# Модель ViT
vit = torchvision.models.vit_b_16(weights='IMAGENET1K_V1')
vit.heads.head = nn.Linear(vit.heads.head.in_features, NUM_CLASSES)
vit.to(device)


# Общая функция обучения
def train(model, trainloader, val_loader, epochs):
    model.train()
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)
    for epoch in range(epochs):
        model.train()
        for imgs, labels in trainloader:
            imgs, labels = imgs.to(device), labels.to(device)
            optimizer.zero_grad()
            out = model(imgs)
            loss = criterion(out, labels)
            loss.backward()
            optimizer.step()

        # Оценка на валидации после каждой эпохи
        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for imgs, labels in val_loader:
                imgs, labels = imgs.to(device), labels.to(device)
                out = model(imgs)
                preds = out.argmax(1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)
        print(f"Epoch {epoch+1}/{epochs}: val acc = {correct/total:.4f}")

# Функция подсчёта метрик
def eval_model(model, testloader):
    model.eval()
    acc_metric = MulticlassAccuracy(num_classes=NUM_CLASSES, average='macro').to(device)
    f1_metric = MulticlassF1Score(num_classes=NUM_CLASSES, average='macro').to(device)
    all_preds, all_labels = [], []
    with torch.no_grad():
        for imgs, labels in testloader:
            imgs, labels = imgs.to(device), labels.to(device)
            out = model(imgs)
            preds = torch.argmax(out, 1)
            acc_metric.update(preds, labels)
            f1_metric.update(preds, labels)
            all_preds.append(preds.cpu())
            all_labels.append(labels.cpu())
    acc = acc_metric.compute().item()
    f1 = f1_metric.compute().item()
    return acc, f1

# Обучение и оценка
print("=== Обучаем ViT ===")
train(vit, trainloader_vit, testloader_vit, EPOCHS)
acc_vit, f1_vit = eval_model(vit, testloader_vit)
print(f"ViT_B_16: Test accuracy: {acc_vit:.4f}, Test Macro F1: {f1_vit:.4f}")


print("=== Обучаем ResNet (CNN) ===")
train(cnn, trainloader_cnn, testloader_cnn, EPOCHS)
acc_cnn, f1_cnn = eval_model(cnn, testloader_cnn)
print(f"ResNet: Test accuracy: {acc_cnn:.4f}, Test Macro F1: {f1_cnn:.4f}")


=== Обучаем ViT ===
Epoch 1/3: val acc = 0.8186
Epoch 2/3: val acc = 0.8560
Epoch 3/3: val acc = 0.9003
ViT_B_16: Test accuracy: 0.9003, Test Macro F1: 0.9016
=== Обучаем ResNet (CNN) ===
Epoch 1/3: val acc = 0.8572
Epoch 2/3: val acc = 0.8780
Epoch 3/3: val acc = 0.8759
ResNet: Test accuracy: 0.8759, Test Macro F1: 0.8757


### 3.	Улучшение бейзлайна

**a. Формулировка гипотез**

Для улучшения базовых моделей воспользуемся следующими гипотезами:

1. Аугментации данных:

    Использование аугментаций (случайное отражение, случайные повороты) при обучении повысит обобщающую способность моделей.

2. Подбор моделей:

    Более глубокие или современные архитектуры сверточных нейронных сетей, такие как ResNet34, могут показать лучшие результаты по сравнению с ResNet18.

3. Подбор гиперпараметров:

    Модификация learning rate и использование scheduler может ускорить сходимость и/или повысить итоговое качество.

4. Использование предобученных весов:

    Использование предобученных на ImageNet весов для моделей (с последующей донастройкой на Fashion-MNIST) позволит добиться лучшей сходимости, несмотря на разницу в доменах данных.

**b. Проверка гипотез**

Исходя из проверки гипотез, улучшенный бейзлайн включает:

1. Использование аугментаций в обучающей выборке.

2. Архитектуры ResNet34 и ViT_B_16 с предобученными весами (transfer learning).

3. Learning rate: 5e-4, scheduler на снижение lr при plateu по валидационной accuracy.

**d/e. Обучение и оценка на улучшенном бейзлайне**

In [46]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torchmetrics.classification import MulticlassAccuracy, MulticlassF1Score

# Аугментации и трансформации
transform_aug_cnn = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
transform_cnn_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

transform_aug_vit = transforms.Compose([
    transforms.Resize(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.repeat(3, 1, 1)),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
transform_vit_test = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.repeat(3, 1, 1)),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

BATCH_SIZE = 128
NUM_CLASSES = 10
EPOCHS = 3
LR = 5e-4
PATIENCE = 2
device = torch.device("mps")

# Датасеты и DataLoader'ы
trainset_cnn = torchvision.datasets.FashionMNIST('data', train=True, download=True, transform=transform_aug_cnn)
testset_cnn  = torchvision.datasets.FashionMNIST('data', train=False, download=True, transform=transform_cnn_test)
trainset_vit = torchvision.datasets.FashionMNIST('data', train=True, download=True, transform=transform_aug_vit)
testset_vit  = torchvision.datasets.FashionMNIST('data', train=False, download=True, transform=transform_vit_test)

trainloader_cnn = torch.utils.data.DataLoader(trainset_cnn, batch_size=BATCH_SIZE, shuffle=True)
testloader_cnn  = torch.utils.data.DataLoader(testset_cnn, batch_size=BATCH_SIZE, shuffle=False)

N_SUBSET = 5000
np.random.seed(42)
indices_train = np.random.choice(len(trainset_vit), N_SUBSET, replace=False)
trainset_vit_small = Subset(trainset_vit, indices_train)

trainloader_vit = torch.utils.data.DataLoader(trainset_vit_small, batch_size=BATCH_SIZE, shuffle=True)
testloader_vit  = torch.utils.data.DataLoader(testset_vit, batch_size=BATCH_SIZE, shuffle=False)


# Создание улучшенных моделей

# ResNet34 с предобученными весами
resnet = torchvision.models.resnet34(weights='IMAGENET1K_V1')
resnet.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
resnet.fc = nn.Linear(resnet.fc.in_features, NUM_CLASSES)
resnet.to(device)

# ViT с предобучением
vit = torchvision.models.vit_b_16(weights='IMAGENET1K_V1')
vit.heads.head = nn.Linear(vit.heads.head.in_features, NUM_CLASSES)
vit.to(device)

# Функции для обучения и оценки
def train(model, trainloader, valloader, epochs):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=PATIENCE, factor=0.5)
    for epoch in range(epochs):
        model.train()
        for imgs, labels in trainloader:
            imgs, labels = imgs.to(device), labels.to(device)
            optimizer.zero_grad()
            out = model(imgs)
            loss = criterion(out, labels)
            loss.backward()
            optimizer.step()

        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for imgs, labels in valloader:
                imgs, labels = imgs.to(device), labels.to(device)
                preds = model(imgs).argmax(1)
                correct += (preds == labels).sum().item()
                total += labels.size(0)
        val_acc = correct/total
        print(f'Epoch {epoch+1}: val_acc={val_acc:.4f}')
        scheduler.step(val_acc)

def eval_model(model, testloader):
    model.eval()
    acc_metric = MulticlassAccuracy(num_classes=NUM_CLASSES, average='macro').to(device)
    f1_metric = MulticlassF1Score(num_classes=NUM_CLASSES, average='macro').to(device)
    with torch.no_grad():
        for imgs, labels in testloader:
            imgs, labels = imgs.to(device), labels.to(device)
            preds = model(imgs).argmax(1)
            acc_metric.update(preds, labels)
            f1_metric.update(preds, labels)
    acc = acc_metric.compute().item()
    f1 = f1_metric.compute().item()
    return acc, f1


# Обучение улучшенных моделей и вывод метрик
print('=== Улучшенный ViT===')
train(vit, trainloader_vit, testloader_vit, EPOCHS)
acc_vit, f1_vit = eval_model(vit, testloader_vit)
print(f'ViT_B_16 improved: Test Accuracy = {acc_vit:.4f}, Macro F1 = {f1_vit:.4f}')

print('=== Улучшенный ResNet ===')
train(resnet, trainloader_cnn, testloader_cnn, EPOCHS)
acc_rn, f1_rn = eval_model(resnet, testloader_cnn)
print(f'ResNet improved: Test Accuracy = {acc_rn:.4f}, Macro F1 = {f1_rn:.4f}')


=== Улучшенный ViT===
Epoch 1: val_acc=0.1967
Epoch 2: val_acc=0.3313
Epoch 3: val_acc=0.2978
ViT_B_16 improved: Test Accuracy = 0.2978, Macro F1 = 0.2708
=== Улучшенный ResNet ===
Epoch 1: val_acc=0.8443
Epoch 2: val_acc=0.8791
Epoch 3: val_acc=0.8814
ResNet improved: Test Accuracy = 0.8814, Macro F1 = 0.8804


**g. Выводы**

1. ResNet правильно реагирует на улучшения: аугментации, увеличение глубины, transfer learning и scheduler даже на таком деперсонализированном датасете как Fashion-MNIST дают хороший, ожидаемый прирост.

2. ViT заметная деградация качества. Возможно сказался высокий LR

### 4. Имплементация алгоритма машинного обучения

**a. Самостоятельно имплементировать модели машинного обучения**
Для сравнения с нейронными сетями рассмотрим:

1. Логистическую регрессию (multinomial)

2. Метод опорных векторов (SVM)

**b/c. Обучение моделей на Fashion-MNIST и оценка качества моделей**

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, f1_score
from sklearn.neighbors import KNeighborsClassifier

import numpy as np
import torchvision
from torchvision import transforms

# Загружаем dataloader с нормализацией как для нейросети
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

trainset = torchvision.datasets.FashionMNIST('data', train=True, download=True, transform=transform)
testset  = torchvision.datasets.FashionMNIST('data', train=False, download=True, transform=transform)

# Преобразуем в numpy
X_train = trainset.data.numpy().reshape(-1, 28*28) / 255.0   # нормируем [0,1]
y_train = trainset.targets.numpy()
X_test  = testset.data.numpy().reshape(-1, 28*28) / 255.0
y_test  = testset.targets.numpy()

N_SUB = 5000
X_train_small = X_train[:N_SUB]
y_train_small = y_train[:N_SUB]

**Логистическая регрессия:**

In [42]:
logreg = LogisticRegression(max_iter=1000, multi_class='multinomial', solver='lbfgs')
logreg.fit(X_train_small, y_train_small)
y_pred_logreg = logreg.predict(X_test)

acc_logreg = accuracy_score(y_test, y_pred_logreg)
f1_logreg = f1_score(y_test, y_pred_logreg, average='macro')
print(f"Логистическая регрессия: accuracy = {acc_logreg:.4f}, macro F1 = {f1_logreg:.4f}")

/Users/rudadadadada/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Логистическая регрессия: accuracy = 0.8112, macro F1 = 0.8118


**SVM:**

In [43]:
svc = LinearSVC(max_iter=2000)
svc.fit(X_train_small, y_train_small)
y_pred_svc = svc.predict(X_test)

acc_svc = accuracy_score(y_test, y_pred_svc)
f1_svc = f1_score(y_test, y_pred_svc, average='macro')
print(f"SVM (Linear): accuracy = {acc_svc:.4f}, macro F1 = {f1_svc:.4f}")

SVM (Linear): accuracy = 0.7867, macro F1 = 0.7867


**e. Выводы**

1. Классические модели (логистическая регрессия, SVM) на Fashion-MNIST показывают качество на уровне 78-81% (accuracy), что весьма достойно для простых моделей без ручных признаков.

2. Глубокие нейронные сети (ResNet18, ViT_B_16) демонстрируют лучшие результаты (accuracy ≈ 87–90%), особенно при большем обучающем датасете и наличии аугментаций.

3. Разрыв в качестве объясняется тем, что нейронные сети могут извлекать более сложные абстрактные признаки, а классические модели работают только с "сырыми" пикселями.

4. Классические модели гораздо быстрее обучаются на CPU на небольшом количестве данных и не требуют GPU.

**f/g/h. Добавить техники из улучшенного бейзлайна и обучить модели, оценка качества моделей**

In [2]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, f1_score
from sklearn.preprocessing import StandardScaler
import torchvision

trainset = torchvision.datasets.FashionMNIST('data', train=True, download=True)
testset  = torchvision.datasets.FashionMNIST('data', train=False, download=True)

X_train = trainset.data.numpy().reshape(-1, 28*28).astype(np.float32) / 255.0
y_train = trainset.targets.numpy()
X_test  = testset.data.numpy().reshape(-1, 28*28).astype(np.float32) / 255.0
y_test  = testset.targets.numpy()

# Стандартизация
scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)

**Логистическая регрессия:**

In [52]:
# 3. GridSearch по C для логрегрессии
param_grid_logreg = {'C': [0.1, 1, 3]}
logreg = LogisticRegression(max_iter=2000, multi_class='multinomial', solver='lbfgs')
gs_logreg = GridSearchCV(logreg, param_grid_logreg, cv=3, n_jobs=1)
gs_logreg.fit(X_train_std, y_train)
print("Лучшие параметры логрегрессии:", gs_logreg.best_params_)
best_logreg = gs_logreg.best_estimator_

y_pred_logreg = best_logreg.predict(X_test_std)
acc_logreg = accuracy_score(y_test, y_pred_logreg)
f1_logreg = f1_score(y_test, y_pred_logreg, average='macro')
print(f"Logistic Regression improved: accuracy = {acc_logreg:.4f}, macro F1 = {f1_logreg:.4f}")

/Users/rudadadadada/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/Users/rudadadadada/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/Users/rudadadadada/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/Users/rudadadadada/Library/Python/3.9/lib/python/site-packages/sklearn/l

Лучшие параметры логрегрессии: {'C': 0.1}
Logistic Regression improved: accuracy = 0.8433, macro F1 = 0.8425


**SVM:**

In [3]:
# 4. GridSearch по C для SVM (LinearSVC)
param_grid_svc = {'C': [1]}
svc = LinearSVC(max_iter=1000)
gs_svc = GridSearchCV(svc, param_grid_svc, cv=2, n_jobs=1)
gs_svc.fit(X_train_std[:5000], y_train[:5000])
print("Лучшие параметры SVM:", gs_svc.best_params_)
best_svc = gs_svc.best_estimator_

y_pred_svc = best_svc.predict(X_test_std)
acc_svc = accuracy_score(y_test, y_pred_svc)
f1_svc = f1_score(y_test, y_pred_svc, average='macro')
print(f"SVM improved: accuracy = {acc_svc:.4f}, macro F1 = {f1_svc:.4f}")

/Users/rudadadadada/Library/Python/3.9/lib/python/site-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/rudadadadada/Library/Python/3.9/lib/python/site-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Лучшие параметры SVM: {'C': 1}
SVM improved: accuracy = 0.7450, macro F1 = 0.7454


/Users/rudadadadada/Library/Python/3.9/lib/python/site-packages/sklearn/svm/_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


**j. Выводы**

1. Стандартизация признаков и подбор C через GridSearchCV позволили повысить качество логистической регресии на 3%, а качество SVM ухудшилось на столько же.

2. Улучшенные логистическая регрессия и SVM минимально сократили разрыв по метрикам с простейшими сверточными сетями, но нейросеть ResNet всё равно опережает их на 5%. Улучшенный ViT демонстрирует худшее качество (возможно, из-за высокого LR).

### Итог:

| Модель            | Baseline Accuracy | Baseline F1 | Improved Accuracy | Improved F1 |
|-------------------|------------------|-------------|-------------------|-------------|
| ViT               | 0.9003           | 0.9016      | 0.2978            | 0.2708      |
| ResNet            | 0.8759           | 0.8757      | 0.8814            | 0.8804      |
| Лог. регрессия    | 0.8112           | 0.8118      | 0.8433            | 0.8425      |
| SVM               | 0.7867           | 0.7867      | 0.7450            | 0.7454      |